In [1]:
import numpy as np
import loader_helper
import nibabel as nii
import torch

In [2]:
input_path = '/home/localadmin/segthor/data/train_resampled/Patient_01/GT.nii.gz'
output_path = './out.nii.gz'

In [3]:
data_header = loader_helper.read_nii_header(input_path)

data = np.array(data_header.get_data())[None,None,:,:,:]
data = torch.Tensor(data)
print(data.shape)

torch.Size([1, 1, 500, 500, 305])


In [25]:
tile_shape = (192,192,192)
center_shape = (16,16,16)
border = (88,88,88)

In [26]:
grid = [int(np.ceil(j / i)) for i,j in zip(center_shape, data.shape[2:])]
print(grid, np.prod(grid))

[32, 32, 20] 20480


In [8]:
output = torch.zeros(data.shape)

for i in range(grid[0]):
    for j in range(grid[1]):
        for k in range(grid[2]):
            index_min, index_max = loader_helper.get_indices(position=(i, j, k), center_shape=center_shape,
                                                                     border=border)
            tile = loader_helper.copy(data=data, tile_shape=tile_shape, index_min=index_min,
                                              index_max=index_max)

            loader_helper.copy_back(data=output, tile=tile, center_shape=center_shape, index_min=index_min,
                                    index_max=index_max, border=border)

In [9]:
output_header = nii.Nifti1Image(output[0,0].numpy().astype(np.uint8), data_header.affine)
nii.save(output_header, output_path)


In [10]:
print(np.all(output == data))

True


In [16]:
print(np.where(data_zeros!=data))
w= np.where(data_zeros!=data)

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [17]:
print(data_zeros[0,0,0],data[0,0,0])

0.037048366 0.037048366
